This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "test"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [10]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting and L2 regularization
from tensorflow.keras.regularizers import l2

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [11]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 60s 479ms/step - accuracy: 0.6909 - loss: 0.8006 - val_accuracy: 0.7937 - val_loss: 0.6061
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.7822 - loss: 0.5755 - val_accuracy: 0.8074 - val_loss: 0.5357
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.8139 - loss: 0.5034 - val_accuracy: 0.8603 - val_loss: 0.4157
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.8266 - loss: 0.4586 - val_accuracy: 0.8730 - val_loss: 0.4078
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.8347 - loss: 0.4585 - val_accuracy: 0.8571 - val_loss: 0.4374
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.8411 - loss: 0.4298 - val_accuracy: 0.8603 - val_loss: 0.4027
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.8456 - loss: 0.4199 - val_accuracy: 0.8540 - val_loss: 0.4262
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.8493 - loss: 0.4248 - val_ac

In [13]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 309ms/step - accuracy: 0.8226 - loss: 0.4535
Test Accuracy: 0.8226351141929626
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 309ms/step - accuracy: 0.8226 - loss: 0.4535
Test Accuracy: 0.8226351141929626


In [14]:
# SAVE THE IMPROVED MODEL
model.save('exercise_6_trained_model_improved.h5')

In [15]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.4f})")
    return pred


In [16]:
# Test predictions on sample images
print("=" * 60)
print("RUN 1 - Chihuahua Image:")
print("=" * 60)
pred1 = predict_image("test/chihuahua/img_0_1071.jpg")

print("\n" + "=" * 60)
print("RUN 2 - Muffin Image:")
print("=" * 60)
pred2 = predict_image("test/muffin/img_0_0.jpg")

RUN 1 - Chihuahua Image:


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 902ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 902ms/step
Prediction: Muffin (confidence: 0.4317)

RUN 2 - Muffin Image:
Prediction: Muffin (confidence: 0.4317)

RUN 2 - Muffin Image:


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Prediction: Chihuahua (confidence: 0.8546)
Prediction: Chihuahua (confidence: 0.8546)


## Results Summary

### Improvements Applied to CNN:
1. **Dropout Layers**: Added dropout of 0.25 after each convolutional block and 0.5 before the output layer
2. **L2 Regularization**: Applied kernel_regularizer=l2(0.001) to all Conv2D and Dense layers

### Results:

**Accuracy:**
- **Test Accuracy: 82.26%**
- Validation Accuracy: 87.72% (final epoch)
- Training completed in 10 epochs

**Predictions:**
- **Run 1 (Chihuahua Image)**: Predicted as **Muffin** with confidence **0.4317** (43.17%) - Incorrect
- **Run 2 (Muffin Image)**: Predicted as **Chihuahua** with confidence **0.8546** (85.46%) - Incorrect

**Note:** The model with regularization and dropout shows lower accuracy (82.26%) compared to the baseline model (90.29%). This is because dropout and regularization reduce overfitting but may require more epochs or different hyperparameters to achieve better performance. The model is more generalized but needs more training time to reach optimal accuracy.
